## 2_Scott Allied health dataset

### Using MATLAB (and Greg's code) to join everything together

In [1]:
T=readtable('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/interim_scott_10k_alliedhealth.csv');
save('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/interim_scott_10k_alliedhealth.mat');

In [2]:
data = load('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/interim_scott_10k_alliedhealth.mat')
head(data.T)
disp(data)

data = struct with fields:
    T: [15733x11 table]

         PTID         RID                       DATA_KEY                      T1_YEAR    T1_MON    T1_DAY      VISDATE      EXAMDATE_4WKS_LATER    EXAMDATE_4WKS_B4                                                                             T1_PATH                                                                                                                                                 MWC1T1_PATH                                                                    
    ______________    ____    ____________________________________________    _______    ______    ______    ___________    ___________________    ________________    _________________________________________________________________________________________________________________________________________________________    ___________________________________________________________________________________________________________________________________________________

    {'012_S_4849'}    4849    {'012_S_4849_ADNI-T1_2012

In [3]:
T = convertvars(data.T,@iscell,'string');
head(T)

        PTID        RID                      DATA_KEY                     T1_YEAR    T1_MON    T1_DAY      VISDATE      EXAMDATE_4WKS_LATER    EXAMDATE_4WKS_B4                                                                            T1_PATH                                                                                                                                               MWC1T1_PATH                                                                   
    ____________    ____    __________________________________________    _______    ______    ______    ___________    ___________________    ________________    _______________________________________________________________________________________________________________________________________________________    _________________________________________________________________________________________________________________________________________________

    "012_S_4849"    4849    "012_S_4849_ADNI-T1_2012-11-05_12_04_44.0" 

In [4]:
%% This has most of Greg's code. I'm not going to vectorise it

basepath = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/'

vars = { ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/PTDEMOG_10Sep2025.csv', {'PTGENDER', 'PTDOB', 'PTEDUCAT'}, {NaN, NaT, NaN}, {[], @(x) datetime(x, 'InputFormat', 'MM/yyyy'),[] }}};

for v=1:numel(vars) %%opens the relevant CSV, and makes sure that VISDATE is a column. If not, then it duplicates EXAMDATE and calls it VISDATE.
    Talli = readtable(vars{v}{1});
    Talli.PTID = string(Talli.PTID);
    if(~ismember('VISDATE', Talli.Properties.VariableNames))
        Talli.VISDATE = Talli.EXAMDATE;
    end
   
    fprintf('joining file %s, rows=%d\n', vars{v}{1}, height(Talli));
    
    varNames = vars{v}{2}; %%selects the appropriate columns from the relevant CSV
    if(~isempty(vars{v}{3}))
        fprintf('(setting types)\n');
        for vf=1:numel(vars{v}{3})
             T{:,varNames{vf}} = repmat(vars{v}{3}{vf}, height(T), 1); %% padding missing values with NaNs or NaTs
        end
    else
        T{:,varNames} = nan(height(T),numel(varNames)); %%also padding missing values, but this is for the case where there a column entirely contains missing values
    end
    
    if(~isempty(vars{v}{4})) %%setting the appropriate date format (e.g., datetime)
        fprintf('(converting)\n');
        for vf=1:numel(vars{v}{4})
            if(~isempty(vars{v}{4}{vf}))
                Talli.(vars{v}{2}{vf}) = vars{v}{4}{vf}(Talli.(vars{v}{2}{vf}));
            end
        end
        fprintf('(done)\n')
    end

    for s=1:height(T)
        Tsub = Talli(Talli.PTID == T.PTID(s),:);
        if(~isempty(Tsub))
            Tsub.datediffs = abs(days( (Tsub.VISDATE - T.VISDATE(s))));
            Tsub = sortrows(Tsub, 'datediffs', 'asc'); % order so that the one nearest the imaging appears 1st
            T(s, varNames) = Tsub(1,varNames);
        end
    end
end

fprintf('Width: %d Height: %d', width(T), height(T))
writetable(T, '/rds/general/project/c3nl_scott_students/ephemeral/sankeith/gregbased_scott10k_alliedhealth.csv')

basepath = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/'

joining file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/PTDEMOG_10Sep2025.csv, rows=6199
(setting types)
(converting)
(done)
Width: 14 Height: 15733

In [ ]:
% now progressively join this table to allied "time varying" data but doing it manually
% first specify pairs of filenames and variables to add
vars = { ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv', {'DIAGNOSIS'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MMSE_28Mar2025.csv', {'MMSCORE'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/GDSCALE_28Mar2025.csv', {'GDTOTAL'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MOCA_30Mar2025.csv', {'MOCA'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/NEUROBAT_28Mar2025.csv', {'LIMMTOTAL', 'CLOCKSCOR', 'LDELTOTAL', 'LDELCUE', 'ANART'}}};

for v=1:numel(vars)
    Talli = readtable(vars{v}{1}); %Talli is a diagnosis/MMSE/allied health CSV
    
    fprintf('manually joining file %s, rows=%d\n', vars{v}{1}, height(Talli));

    varNames = vars{v}{2};
    T{:,varNames} = nan(height(T),numel(varNames));

    if(~ismember('VISDATE', Talli.Properties.VariableNames))
        Talli.VISDATE = Talli.EXAMDATE;
    end

    % subset based on matching subject
    for s=1:height(T)
        Tsub = Talli(Talli.PTID == T.PTID(s),:); % Tsub is a subset table of Talli and T, where they have the same patients. But, this match can include multiple visits from a patient.
        fprintf('Width: %d Height: %d', width(Tsub), height(Tsub))
        datediffs = abs(days( (Tsub.VISDATE - T.VISDATE(s))));
        idx = find(datediffs < 28, 1, 'first');

        if(~isempty(idx))
            %Talli2 = [ Talli2; Tsub(idx,:) ];
            T{s, varNames} = Tsub{idx,varNames};
        end
    end
end

% Some post processing
T.PTAGE = years(T.VISDATE - (T.PTDOB + days(14))); % make an estimated age for each

manually joining file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv, rows=15674
Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 2Width: 42 Height: 2Width: 42 Height: 2Width: 42 Height: 2Width: 42 Height: 8Width: 42 Height: 8Width: 42 Height: 8Width: 42 Height: 8Width: 42 Height: 8Width: 42 Height: 8Width:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 1Width: 42 Height: 3Width: 42 Height: 3Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 3Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 7Width: 42 Height: 7Width: 42 He

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 1Width: 42 Height: 1Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 10Width: 42 Height: 10Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 6Width: 42 Height: 2Width: 42 Height: 1Width: 42 Height: 1Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 5Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4Width: 42 Height: 4

...

 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 1Width: 58 Height: 1Width: 58 Height: 5Width: 58 Height: 5Width: 58 Height: 5Width: 58 Height: 5Width: 58 Height: 5Width: 58 Height: 5Wi

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



7Width: 58 Height: 1Width: 58 Height: 1Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 4Width: 58 Height: 1Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 6Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 8Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 7Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 Height: 3Width: 58 H

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 13Width: 32 Height: 3Width: 32 Height: 3Width: 32 Height: 5Width: 32 Height: 5Width: 32 Height: 5Width: 32 Height: 5Width: 32 Height: 5Width: 32 Height: 5Width: 32 Height: 6Width: 32 Height: 6Width: 32 Height: 6Width: 32 Height: 6Width: 32 Height: 6Width: 32 Height: 6Width: 32 Height: 3Width: 32 Height: 3Width: 32 Height: 3Width: 32 Height: 1Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 4Width: 32 Height: 2Width: 32 Height: 2Width: 32 Height: 2Width: 32 Height: 1Width: 

In [12]:
%% Some more post-processing (keeping this separate for 'damage control')

[~, idx] = unique(T.T1_PATH, 'stable');
T = T(idx, :);

fprintf('Width: %d Height: %d', width(T), height(T))

Width: 24 Height: 15733

In [43]:
disp('Exporting...');

% The combined table
writetable(T, '/rds/general/project/c3nl_scott_students/ephemeral/sankeith/gregbased_scott10k_alliedhealth.csv')

save('/rds/general/project/c3nl_scott_students/ephemeral/sankeith/gregbased_scott10k_alliedhealth.mat', 'T')

disp('Done!')

Exporting...
Done!


In [ ]:
% now progressively join this table to allied "time varying" data but doing it manually
% rather than using join functions
% first specify pairs of filenames
data = load

vars = { ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MMSE_28Mar2025.csv', {'MMSCORE'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/DXSUM_10Sep2025.csv', {'DIAGNOSIS'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/GDSCALE_28Mar2025.csv', {'GDTOTAL'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/MOCA_30Mar2025.csv', {'MOCA'}}, ...
    {'/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_extra_adni_info/NPIQ_28Mar2025.csv'{'NPISCORE'}}
};

for v=1:numel(vars)
    Talli = readtable(vars{v}{1});
    
    fprintf('manually joining file %s, rows=%d\n', vars{v}{1}, height(Talli));

    varNames = vars{v}{2};
    T{:,varNames} = nan(height(T),numel(varNames));

    if(~ismember('VISDATE', Talli.Properties.VariableNames))
        Talli.VISDATE = Talli.EXAMDATE;
    end
    %T = innerjoin(T, Talli(:,['PTID', 'VISDATE', vars{v}{2}]), 'LeftKeys', {'PTID', 'imagingdate'}, 'RightKeys', {'PTID', 'VISDATE'});
    %fprintf('joined file %s, height=%d\n', vars{v}{1}, height(T));

    % subset based on matching subject
    Talli2 = table;

    for s=1:height(T)
        Tsub = Talli(Talli.PTID == T.PTID(s),:);
        datediffs = abs(days( (Tsub.VISDATE - T.VISDATE(s))));
        idx = find(datediffs < 28, 1, 'first');

        if(~isempty(idx))
            %Talli2 = [ Talli2; Tsub(idx,:) ];
            T{s, varNames} = Tsub{idx,varNames};
        end
    end
end